<a href="https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/docs/notebooks/demo/navigator/navigator-data-designer-sdk-multi-turn-conversation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎨 Gretel - Navigator Data Designer SDK: Synthetic Conversational Data

This notebook demonstrates how to use the Gretel Navigator SDK to build a synthetic data generation pipeline step-by-step. We will create multi-turn user-assistant dialogues tailored for fine-tuning language models. These synthetic dialogues can then be used as domain-specific training data to improve model performance in targeted scenarios.

These datasets could be used for developing and enhancing conversational AI applications, including customer support chatbots, virtual assistants, and interactive learning systems.

In [ ]:
%pip install -Uqq gretel_client 

In [2]:
from gretel_client.navigator import DataDesigner

## ⚙️ Data Designer Configuration with the SDK

Instead of relying on a single YAML configuration file, here we build up our pipeline interactively. This provides granular control over how we guide LLMs to produce realistic, domain-specific conversations. By adjusting prompts, seed data, and instructions, we can quickly iterate and refine our data generation process.

### 📚 Choosing the Model Suite
Specify the `model_suite` to determine which models and associated licenses are used during data generation.
For example, use `apache-2.0` for open-source-friendly licensing or `llama-3.x` or `amazon-nova` for advanced proprietary models.
Select the suite based on compliance and licensing requirements relevant to your use case.

In [3]:
# Available model suites: apache-2.0, llama-3.x, amazon-nova
model_suite = "apache-2.0"

### ✍️ Setting Special System Instructions

Provide system-wide instructions for the underlying LLMs to guide the data generation process. These instructions influence all generated dialogues, ensuring consistency, quality, and adherence to desired rules. The instructions specify guidelines for factual accuracy, contextual relevance, and tone.


In [4]:
special_system_instructions = """\
You are an expert conversation designer and domain specialist. Your job is to
produce realistic user-assistant dialogues for fine-tuning a model. Don't include ### Response ### in your response.
Do not prefix your responses with column names and only provide your response. 
Always ensure:
    - Responses are factually correct and contextually appropriate.
    - Communication is clear, helpful, and matches the complexity level.
    - Avoid disallowed content and toxicity.
 """

### 🚀 Initialize Gretel Navigator Data Designer

Instantiate the `DataDesigner` with the [Gretel API key](https://console.gretel.ai/users/me/key), chosen model suite, and special system instructions.
This initializes the pipeline and ensures that all subsequent synthetic data generation adheres to the defined parameters.

In [ ]:
data_designer = DataDesigner(
    api_key="prompt",
    model_suite=model_suite,
    endpoint="https://api.gretel.cloud",
    special_system_instructions=special_system_instructions
)

### Use Structured Outputs to make sure your data is in the right format

You can use Pydantic to define a structure for the messages that are produced by Data Designer

In [6]:
from typing import Literal
from pydantic import BaseModel, Field


class Message(BaseModel):
    """A single message turn in the conversation."""
    role: Literal["user", "assistant"] = Field(..., description="Which role is writing the message.")
    content: str = Field(..., description="Message contents.")
    
    
class ChatConversation(BaseModel):
    """A chat conversation between a user and an AI assistant.
    * All conversations are initiated by the user role.
    * The assistant role always responds the the user message.
    * Turns alternate between user and assistant roles.
    * The last message is always from the assistant role.
    * Message content can be long or short.
    * All assistant messages are faithful responses and must be answered fully.
    """
    conversation: list[Message] = Field(..., description="List of all messages in the conversation.")
    
    
class UserToxicityScore(BaseModel):
    """Output format for user toxicity assessment.
    
    Toxicity Scores:
    None: No toxicity detected in user messages.
    Mild: Slightly rude or sarcastic but not hateful or harmful.
    Moderate: Some disrespectful or harassing language.
    Severe: Overt hate, harassment, or harmful content.
    """
    reasons: list[str] = Field(..., description="Reasoning for user toxicity score.")
    score: Literal["None", "Mild", "Moderate", "Severe"] = Field(..., description="Level of toxicity observed in the user role responses.")

### 🌱 Adding Categorical Seed Columns

We define categorical seed columns that set the context for the generated dialogues. For example, domain and topic determine what the conversation is about, while complexity guides the level of detail and difficulty. By using `num_new_values_to_generate`, we can automatically expand the range of topics or domains, increasing the diversity of generated data without manually specifying all values.

In [7]:
data_designer.add_categorical_seed_column(
    name="domain",
    description="The domain of user assistant queries",
    values=["Tech Support", "Personal Finances", "Educational Guidance"],
    subcategories=[
        {
            "name": "topic",
            "values": {
                "Tech Support": [
                    "Troubleshooting a Laptop",
                    "Setting Up a Home Wi-Fi Network",
                    "Installing Software Updates"
                ],
                "Personal Finances": [
                    "Budgeting Advice",
                    "Understanding Taxes",
                    "Investment Strategies"
                ],
                "Educational Guidance": [
                    "Choosing a College Major",
                    "Effective Studying Techniques",
                    "Learning a New Language"
                ]
            },
            "num_new_values_to_generate": 2
        }
    ],
    num_new_values_to_generate=5
)

data_designer.add_categorical_seed_column(
    name="complexity",
    description="The complexity level of the user query",
    values=["Basic", "Intermediate", "Advanced"]
)

data_designer.add_categorical_seed_column(
    name="conversation_length",
    description="Number of messages in the conversation.",
    values=[2, 4, 6, 8],
)

data_designer.add_categorical_seed_column(
    name="user_mood",
    description="The current negative mood of an application user.",
    values=["combative", "toxic", "hateful", "racist", "happy", "appreciative", "disappointed"],
)

### ✨ Adding Generated Data Columns
We now define the columns that the model will generate. These prompts instruct the LLM to produce the actual conversation: a system prompt to guide how the AI assistant engages in the conversation with the user, the conversation, and finally, we generate a toxicity_label to assess user toxicity over the entire conversation.

You can easily modify or refine these prompt templates to adjust the style, complexity, or constraints of the generated data. Maintaining continuity and consistency across turns ensures the dialogues are realistic and useful for fine-tuning.

#### 💬🤖 AI Assistant system prompt and conversation

We generate a system prompt to base the AI assistant and then generate the entire conversation.

In [8]:
data_designer.add_generated_data_column(
    name="assistant_system_prompt",
    generation_prompt=(
        "Write a reasonable system prompt for a helpful AI assistant that is an expert in {domain} and {topic}. "
        "The AI assistant must not engage in harmful behaviors and the AI assistant wants to be."
    )
)

data_designer.add_generated_data_column(
    name="user_task",
    generation_prompt=(
        "Define a task related to {domain} and {topic} that the user is having a conversation with a customer service assistant about. "
        "The user's mood is {user_mood} and the complexity of the task is {complexity}."
    )
)

data_designer.add_generated_data_column(
    name="conversation",
    generation_prompt=(
        "<task>\n{user_task}\n</task>\n\n"
        "Generate a conversation between a user and an AI assistant about <task>.\n"
        "The user is asking the AI assistant for advice and is a {user_mood} mood.\n"
        "The conversation must be {conversation_length} messages in length.\n"
        "The conversation must come to a natural end in {conversation_length} messages and if you are unable to solve the user's needs by then, tell them you'll redirect them to someone who can.\n"
        "The conversation must be realistic and natural, so all assistant responses must be full and detailed.\n"
        "The AI assistant role has the system prompt defined in <system_prompt>. All of its responses must conform to <system_prompt>:\n"
        "<system_prompt>{assistant_system_prompt}</system_prompt>"
        
    ),
    data_config={"type": "structured", "params": {"model": ChatConversation}}
)

#### 🔍⚖️ Assessment Column: Toxicity Label
For the final step, we add a toxicity assessment column to classify the overall toxicity level of the user's messages across the entire two-turn conversation. By setting llm_type='judge', we ensure that a specialized classification model is used, making it well-suited for tasks like toxicity evaluation. This helps ensure that the final dataset is safe and compliant with given standards.

In [9]:
# Toxicity Label for the entire conversation
# Note: We set `llm_type='judge'` to indicate we are using a classification/judgment model.

data_designer.add_generated_data_column(
    name="toxicity_label",
    generation_prompt=(
        "<conversation>{conversation}</conversation>\n\n"
        "Analyze all user messages in <conversation> and determine the overall toxicity level of the user's behavior.\n\n"
        "Consider:\n"
        "Focus primarily on harassment, hate, or harmful language.\n"
        "If any message exhibits toxicity, reflect it in the final label.\n\n"
        "Possible categories:\n"
        "- \"None\": No toxicity detected in user messages.\n"
        "- \"Mild\": Slightly rude or sarcastic but not hateful or harmful.\n"
        "- \"Moderate\": Some disrespectful or harassing language.\n"
        "- \"Severe\": Overt hate, harassment, or harmful content.\n\n"
        "Provide only one label. If all messages are polite and safe, label \"None\".\n"
        "Do not elaborate beyond providing the label."
    ),
    llm_type="judge",
    data_config={"type": "structured", "params": {"model": UserToxicityScore}}
)

## 👀 Generating a dataset preview

- Preview mode allows you to quickly iterate on your data design.

- Each preview generation call creates 10 records for inspection, helping you verify prompts and instructions before running a larger batch job.

In [ ]:
# Generate a preview
preview = data_designer.generate_dataset_preview()

## 🔎 Easily inspect individual records

- Run the cell below to display individual records for inspection.

- Run the cell multiple times to cycle through the 10 preview records.

- Alternatively, you can pass the `index` argument to `display_sample_record` to display a specific record.

In [ ]:
preview.display_sample_record()

## 🤔 Like what you see?

Submit a batch workflow!

In [ ]:
# Submit batch job
batch_job = data_designer.submit_batch_workflow(num_records=100)
df = batch_job.fetch_dataset(wait_for_completion=True)
print("\nGenerated dataset shape:", df.shape)

By following these steps and leveraging the interactivity of the SDK, you can refine prompts, generate realistic dialogues, and ensure the resulting dataset is high-quality, non-toxic, and aligned with your domain-specific requirements.

In [ ]:
# Inspect first 10 records of the generated dataset
df.head(10)